In [ ]:
library(dplyr)

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
str(df)

In [ ]:
head(df)

In [ ]:
# # Remove extraneous variables 

# df$X <- NULL
# df$X.1 <- NULL
# df$X.2 <- NULL
# head(df, 3)

### <font color = lime> Convert the df char data types to numer_remainderic

In [ ]:
# Convert Excel dates when in the 1900 date format
df$date <- as.Date(df$date, origin = "1899-12-30")
df$date_s <- as.Date(df$date_s, origin = "1899-12-30")
head(df, 3)

In [ ]:
# create a test loop for moving through the df
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
# Define the target sum
target_value <- 900
master_ct <- 0

# Initialize a variable to hold the increasing cumulative sum
cumulative_sum <- 0

# Initialize a variable to hold the index where target sum is achieved
index <- 0

# Iterate over the column
  for(i in 1:nrow(df)) {
  # counter at top of loop (will always be >= 1)
  master_ct <- master_ct + 1
  # creates a tally value of quan values from each row that is compared against the target value  
  cumulative_sum <- cumulative_sum + df[i, 'quan']
    print(cumulative_sum)
  # If the cum_sum equals the target value, change nothing
  if(cumulative_sum == target_value) {
      temp <- df[i, 'tally_q']
      df[i, 'tally_q'] <- temp
      break
      }
  # if the cum_sum exceeds the target value, break and return the index (count of rows for the if )
  else if(cumulative_sum >= target_value) {
    # count the times when the cumulative_sum greater than target value. Can be zero.  
    index <- i     
    break
  }
}

# Print the indexes
print(paste0("the master counter value is: ", master_ct))
print(paste0("the incrementing index value is: ", index))

# print the cumulative sum at the break out from the loop
cumulative_sum

In [ ]:
index

In [ ]:
# Assuming 'df' is your dataframe and 'Quantity' is your column
df <- data.frame(Quantity = c(1, 2, 3, 4, 5), Other = c('a', 'b', 'c', 'd', 'e'))
test_values <- c(8, 6)
df
# Iterate over the test values
for(test_value in test_values) {
  # Initialize a variable to hold the cumulative sum
  cumulative_sum <- 0

  # Iterate over the rows
  for(i in 1:nrow(df)) {
    cumulative_sum <- cumulative_sum + df[i, 'Quantity']
    
    # Check if the cumulative sum exceeds the test value
    if(cumulative_sum > test_value) {
      # Calculate the overage and the adjusted quantity
      overage <- cumulative_sum - test_value
      remainder <- df[i, 'Quantity'] - overage
      
      # Update the quantity in the current row
      df[i, 'Quantity'] <- remainder
      
      # Create a new row with the overage quantity
      new_row <- df[i, ]
      new_row['Quantity'] <- overage
      
      # Insert the new row into the dataframe immediately below the current row
      df <- df %>% 
        add_row(.before = i+1, !!!new_row)
      
      # Break the loop
      break
    }
  }
}

# Print the updated dataframe
print(df)


## <font color = lime> Keeper!  It works

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
# df$X <- NULL
# df$X.1 <- NULL
# df$X.2 <- NULL



ct <<- 1
test_value <- c(850)
rat1 <<- 0

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {
      
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    
    # Check if the cumulative sum exceeds the test value
    if(cumulative_sum == test_value) {
        print("match amount")       
        }
      
    if(cumulative_sum >= test_value) {        
           
      # Calculate the overage and the adjusted quantity
      overage <- cumulative_sum - test_value         # 900 - 700 = 200
      remainder <- df[i, 'quan'] - overage   # deduct 200 from existing value 
       
     # establish the ratio - need for row 1 and row 2   
     denom_quan <- df[i, 'quan']    # 400    
     numer_remainder <- remainder
      
      # Update the quantity in the current row
      df[i, 'quan'] <- remainder            # assign new value to quan
        
      # Create a new row with the overage quantity (Do it here and not below the changes)
      new_row <- df[i, ]
      new_row['quan'] <- overage 
      
    
    # update the tally_q (first row)
      q1 <- df[i, 'tally_q'] - remainder
      df[i, 'tally_q'] <- q1
        
    
   # first, establish a ratio to apply to variables in row 1, then allocate the variables
       
    # create the ratio
    rat1 <<- numer_remainder / denom_quan
               
    # apply ratio to variables in row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i-1,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i-1, 'tally_q2'] - remainder
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i-1, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        
        
    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1        

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
    t_c2 <- df[ct-1,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
    t_q22 <- df[ct-1, 'tally_q2'] - remainder
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
    t_c22 <- df[i-1, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22      

  # Break the loop and move to the next test value
    ct <- ct + 1
  break
    
    }
    
  }
     return(df)
}


print(ct)
# make fct call
newdf <- inter(df, test_value, ct) 
tail(newdf, 3)


# Remove the unwanted row of NAs
newdf <- newdf[complete.cases(newdf), ]  
tail(newdf, 3)



# show the returned ct value


In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
ct <<- 1
ct2 <<- 2

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,
           500,2000,3000,500,2000,810,1000,190,3000,1000,1100)
length(sales)

for(i in 1:length(sales)-1) {
    mod_df <- inter(df, sales[ct], ct)
    # okay ... prints 22 times the length of sales 
    
    #print(mod_df)
    ct <<- ct + 1
    # dont need the following code
    # df <- df[complete.cases(df), ]
    # ct <- ct + 1
    
    ct2 <- i + 1
    }    
ct
ct2
tail(mod_df,15)

In [ ]:

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,
           3000,500,2000,3000,500,2000,810,1000,190,3000,1000,1100)

ct <- 0

for(i in 1:length(sales)) {
    #tally sales
    ct <<- ct + sales[i]
    }    
ct



In [ ]:
# This demonstrates preventing the loop exceeding the index that creates errors

df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")

ct <- 0

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,
           3000,3000,500,2000,3000,500,2000,810,1000,190,3000,1000,1100)

length(sales)
str(sales)

start_row <- 1

for(i in start_row:nrow(df)) {
        
    if(df[i, 'quan'] <= sales[i])  {
        temp <- df[i, 'quan'] 
        df[i,'quan'] <- temp
        ct <<- ct + sales[i]
       # print(paste0("ct is ", ct))
    } else {
        ct <<- ct + sales[i]
        # print(paste0("ct is ", ct))
        }
    
    # To prevent going past index by 1 
    start_row <- start_row + 1
    if(start_row > length(sales) ) {
        break
    }
 }
 
#tally of ct
ct

In [ ]:
# demonstrates looping 10 times

sales <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,
           500,2000,3000,500,2000,810,1000,190,3000,1000,1100)

ct <- 0

length(sales)
# str(sales)
tally <- 0
start_row <- 1

for(i in 1:length(df))  { 
    
    tally <- tally + 1 
    
    # demonstrate rplacing values in the df$quan for each item in sales vector
    temp <- df[i, 'quan'] 
    # demonstrates the changes to quan
    print(paste0("value of temp is ", temp))
    # demonstrates looping through the sales vector
    print(sales[i])
    
#   if(cumulative_sum >= test_value) {         
    if(temp <= sales[i])  {
        # add 10 to each existing quan value
        df[i,'quan'] <- temp + 10
        ct <<- ct + 1
    } 
    else {
      ct <<- ct + 1
        }
    start_row <- start_row + 1
    print(paste0("count is: ", ct))
}
 
print(paste0("The tally is :", tally))
ct
head(df,3)

In [ ]:
# No errors, but nothing happened to the df

df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")

cumulative_sum <- 0
ct <<- 0
start_row <- 1

#Iterate over the sales values
for(value in sales) {
    ct <<- ct + 1
    # Iterate over the rows
    for(i in start_row:nrow(df)) {
        cumulative_sum <- 0
        cumulative_sum <- cumulative_sum + df[ct, 'quan']
        print(cumulative_sum)
        
        # check if the cumulative sum equals the test value in sales[i], if so, do nada
        if(cumulative_sum <= sales[i]) {
            temp <- df[i, 'quan']
            df[i, 'quan'] <- temp 
            start_row <- i + 1
            print(ct)
            break
        }
        # Check if the cumulative sum exceeds the test value
         if(cumulative_sum >= sales[i]) {        
    
            # Calculate the overage and the adjusted quantity
            overage <- cumulative_sum - test_value         # 900 - 700 = 200
            remainder <- df[i, 'quan'] - overage   # deduct 200 from existing value 

            # establish the ratio - need for row 1 and row 2   
            denom_quan <- df[i, 'quan']    # 400    
            numer_remainder <- remainder

            # Update the quantity in the current row
            df[i, 'quan'] <- remainder            # assign new value to quan

            # Create a new row with the overage quantity (Do it here and not below the changes)
            new_row <- df[i, ]
            new_row['quan'] <- overage  

            start_row <- i + 1
            break
        }
      }
    }
ct
nrow(df)
head(df,4)

In [ ]:
# Assuming 'vec' is your vector and 'df' is your dataframe
vec <- sales
start_row <- 1
for(value in vec) {
  for(i in start_row:nrow(df)) {
    if(df[i, 'var'] == value) {  # Replace 'var' with your actual column name
      # Execute your code here
      print(paste("Match found for", value, "at row", i))
      start_row <- i + 1
      break
    }
  }
}


In [ ]:
stopper


## <font color = lime> Keeper Below: Part 2
    
<div style="margin-left:40px;">
Allocate the other data in the matched row
</div> 


In [ ]:
# The function allocated the quan in the row of the match with test_value  = 850
# Rows 3 and 4 modified / added new row for the allocation of quan 


df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")

ct <<- 0
test_value <- c(850)
rat1 <<- 0

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {
      
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    
    # Check if the cumulative sum exceeds the test value
    
      
    if(cumulative_sum >= test_value) {
        
      # Calculate the overage and the adjusted quantity
      overage <- cumulative_sum - test_value         # 900 - 700 = 200
      remainder <- df[i, 'quan'] - overage   # deduct 200 from existing value 
       
     # establish the ratio - need for row 1 and row 2   
     denom_quan <- df[i, 'quan']    # 400    
     numer_remainder <- remainder
      
      # Update the quantity in the current row
      df[i, 'quan'] <- remainder            # assign new value to quan
        
      # Create a new row with the overage quantity (Do it here and not below the changes)
      new_row <- df[i, ]
      new_row['quan'] <- overage 
      
    
    # update the tally_q (first row)
      q1 <- df[i, 'tally_q'] - remainder
      df[i, 'tally_q'] <- q1
        
    
   # first, establish a ratio to apply to variables in row 1, then allocate the variables
       
    # create the ratio
    rat1 <<- numer_remainder / denom_quan
               
    # apply ratio to variables in row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i-1,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i-1, 'tally_q2'] - remainder
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i-1, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        
        
    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1        

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
    t_c2 <- df[ct-1,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
    t_q22 <- df[ct-1, 'tally_q2'] - remainder
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
    t_c22 <- df[i-1, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22      

  # Break the loop and move to the next test value
  break
    
    }
    
  }
     return(df)
}

# make fct call

mod_df <- inter(df, test_value, ct)

# show the returned ct and ratio values
ct
rat1

head(mod_df,10)



## <font color = lime> The Keeper! This works great! Not actual sales values
    

In [ ]:
# Moving towards.  Creates a new df from the below the match / new rows 
#  and eliminates some rows to run the process again


df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
df1 <- data.frame()

ct <<- 0
test_value <- c(850)
rat1 <<- 0

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {  
      
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    
    # Check if the cumulative sum exceeds the test value
    
      
    if(cumulative_sum >= test_value) {
        
      # Calculate the overage and the adjusted quantity
      overage <- cumulative_sum - test_value         # 900 - 700 = 200
      remainder <- df[i, 'quan'] - overage   # deduct 200 from existing value 
       
     # establish the ratio - need for row 1 and row 2   
     denom_quan <- df[i, 'quan']    # 400    
     numer_remainder <- remainder
      
      # Update the quantity in the current row
      df[i, 'quan'] <- remainder            # assign new value to quan
        
      # Create a new row with the overage quantity (Do it here and not below the changes)
      new_row <- df[i, ]
      new_row['quan'] <- overage 
      
    
    # update the tally_q (first row)
      q1 <- df[i, 'tally_q'] - remainder
      df[i, 'tally_q'] <- q1
        
    
   # first, establish a ratio to apply to variables in row 1, then allocate the variables
       
    # create the ratio
    rat1 <<- numer_remainder / denom_quan
               
    # apply ratio to variables in row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i-1,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i-1, 'tally_q2'] - remainder
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i-1, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        
        
    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1  
    # Insert the new row into df1
    df1 <<- df %>% 
    add_row(.before = i+1, !!!new_row)

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
#   t_c2 <- df[ct-1,'tally_c'] + c2
    # Correction? 
    t_c2 <- df[ct,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
#t_q22 <- df[ct-1, 'tally_q2'] - remainder    
    # Correction ??
    t_q22 <- df[ct, 'tally_q2'] - remainder
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
#    t_c22 <- df[i-1, 'tally_c2'] - c2
    # Correction ???
    t_c22 <- df[i, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22 
      
     # build df1
     df1 <<- rbind(df1, df[i,])

  # Break the loop and move to the next test value
  break
    
    }
    
  }
     return(df)
}

In [ ]:
stopper


## <font color = lime> Real Test - Real Sales Values

### <font color = lime> Clean Function


In [ ]:
# clean df
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")
# clean sales vector
sales <- c(1000,3000,3000,6000,5850,3000,6000,2000,5000,1000,1100)

In [ ]:
length(sales)
sum(sales)

In [ ]:
df

In [ ]:
# S/B a clean funtion

ct <<- 0
rat1 <<- 0
cumVal <<- as.vector(c())  # for debugging / solutions

# function to allocate values within a df
inter <- function(df, test_value, ct) {
     cumulative_sum <- 0
      
# Iterate over the rows
  for(i in 1:nrow(df)) {
    
   
     # assign quan value to cum_sum 
    cumulative_sum <- cumulative_sum + df[i, 'quan']
    #assign valu to tracking vector
    cumVal <- cumulative_sum
      
          
    
    # Check if the cumulative sum exceeds the test value
    
    # when the sum of quan values exceeds the amount fof the sales value  
    if(cumulative_sum >= test_value) {        
        
      # Calculate the overage and the adjusted quantity
      overage <- cumulative_sum - test_value         # 3000 - 190 = 2810
      # deduct the overage from the current quan value  # 190 - 2810   
      remainder <- df[i, 'quan'] - overage   
       
     # establish the ratio - needed to allocate values for row 1 and the new row 2   
      numer_remainder <- remainder  # 300
      denom_quan <- df[i, 'quan']      # 400  
     # create the ratio for row1
      rat1 <<- numer_remainder / denom_quan
      
      # Update the quan value in the row1
      df[i, 'quan'] <- remainder           
        
      # Create new row, row2, with the overage value 
      new_row <- df[i, ]
      new_row['quan'] <- overage       
    
    # update the tally_q (row1)
      q1 <- df[i, 'tally_q'] - remainder
      df[i, 'tally_q'] <- q1
        
        
        
    # Insert the new row into the dataframe immediately below the current row
    # df <- df %>% 
    # add_row(.before = i+1, !!!new_row)
    # ct <<- i+1  
    # # Insert the new row into df1
    # df1 <<- df %>% 
    # add_row(.before = i+1, !!!new_row)   
        
        
        
    
   #  Apply the ratio, rat1, to allocate the variables on row 1
    # cost var
    c1 <- df[i, 'cost'] * rat1
    df[i, 'cost'] <- c1  
    #tally_c var
    t_c <- df[i,'tally_c'] + c1
    df[i, 'tally_c'] <- t_c
    #proceeds var
    p1 <- df[i, 'proceeds'] * rat1
    df[i, 'proceeds'] <- p1
    # t_q2 var
    t_q2 <- df[i, 'tally_q2'] - remainder
    df[i, 'tally_q2'] <- t_q2
    # t_c2 var
    t_c2 <- df[i, 'tally_c2'] - c1
    df[i, 'tally_c2'] <- t_c2
        

    # Insert the new row into the dataframe immediately below the current row
    df <- df %>% 
    add_row(.before = i+1, !!!new_row)
    ct <<- i+1  
    # Insert the new row into df1
    df1 <<- df %>% 
    add_row(.before = i+1, !!!new_row)

#  PART 2
        
   # update the values in new row 2 
        
    # cost var
    c2 <- df[ct, 'cost'] * rat1
    df[ct, 'cost'] <- c2  
    #tally_c var
    t_c2 <- df[ct-1,'tally_c'] + c2
    df[ct, 'tally_c'] <- t_c2
    #proceeds var
    p2 <- df[ct, 'proceeds'] * rat1
    df[ct, 'proceeds'] <- p2
      # t_q2 var
    t_q22 <- df[ct-1, 'tally_q2'] - remainder
    df[ct, 'tally_q2'] <- t_q22
    # t_c2 var
    t_c22 <- df[i-1, 'tally_c2'] - c2
    df[i, 'tally_c2'] <- t_c22 
      
     # build df1
     df1 <<- rbind(df1, df[i,])

  # Break the loop and move to the next test value
  break
    
    }
    
  }
     return(df)
}


In [ ]:
head(df)

In [ ]:
# First value from sales passed to fct 
test_value <- sales[1]

In [ ]:
df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")

In [ ]:
# make fct call 1  (sales 1000)

mod_df <<- inter(df, test_value, ct)
nrow(mod_df)

# show the returned ct value
ct
rownum <- ct
rat1

# create portions of the df
df1a <- df1[1:ct, ]
df <- df[ct:nrow(df), ]


# print the modified df, and portions of the df
head(mod_df)
nrow(mod_df)

print(df1a)  # This is good.
nrow(df1a)

head(df)  # this is good
nrow(df)
length(sales) # sale[] = 22 == 1000

In [ ]:
ct

In [ ]:
# make fct call 2  (sales 1000)

# remove previous sale value from vector
sales <- sales[2:length(sales)]
# assign new sale value to test_value
test_value <- sales[1]
# zero ct
ct <<- 0

mod_df <<- inter(df, test_value, ct)
nrow(mod_df)

# show the returned ct value
ct
rownum <- ct
rat1

# create portions of the df
df1a <- df1[1:ct, ]
df <- df[ct:nrow(df), ]
#sales <- sales[2:length(sales)]

# print the modified df, and portions of the df
head(mod_df)
nrow(mod_df)

print(df1a)
nrow(df1a)

head(df)
nrow(df)
length(sales)   # sale[] = 21

In [ ]:
length(sales)

In [ ]:
# make fct call 3 (sales 1000)

# remove previous sale value from vector
sales <- sales[2:length(sales)]
# assign new sale value to test_value
test_value <- sales[1]
# zero ct
ct <<- 0

mod_df <<- inter(df, test_value, ct)
nrow(mod_df)

# show the returned ct value
ct
rownum <- ct
rat1

# create portions of the df
df1a <- df1[1:ct, ]
df <- df[ct:nrow(df), ]
#sales <- sales[2:length(sales)]

# print the modified df, and portions of the df
head(mod_df)
nrow(mod_df)

print(df1a)
nrow(df1a)

head(df)
nrow(df)
length(sales)  #  sale[] = 20

In [ ]:
cumulative_sum
test_value
cumVal

In [ ]:
# make fct call 4 (sales 100)

# remove previous sale value from vector
sales <- sales[2:length(sales)]
# assign new sale value to test_value
test_value <- sales[1]
# zero ct
ct <<- 0

mod_df <<- inter(df, test_value, ct)
nrow(mod_df)

# show the returned ct value
ct
rownum <- ct
rat1

# create portions of the df
df1a <- df1[1:ct, ]
df <- df[ct:nrow(df), ]
#sales <- sales[2:length(sales)]

# print the modified df, and portions of the df
head(mod_df)
nrow(mod_df)

print(df1a)
nrow(df1a)

head(df)
nrow(df)
length(sales)  # sale[] = 19

In [ ]:
stopper

### <font color = yellow> Early practive below

In [ ]:
# Another issue with exceeding the index

df <- data.frame(quan = c(1,2,3,4,5,6,7,8,9,10))
vec <- c(6, 15, 28)

# Initialize variables
start_index <- 1
vec_index <- 1

# Loop over the vector values
for (target in vec) {
  # Initialize sum
  sum_quan <- 0
  
  # Loop over the dataframe rows
  for (i in start_index:nrow(df)) {
    # Add the current 'quan' value to the sum
    sum_quan <- sum_quan + df$quan[i]
    
    # Check if the sum exceeds the current target value
    if (sum_quan > target) {
      # Calculate the overage value
      overage <- sum_quan - target
      
      # Adjust the 'quan' value in the current row
      df$quan[i] <- df$quan[i] - overage
      
      # Create a new row with the overage value
      new_row <- df[i, ]
      new_row$quan <- overage
      df <- rbind(df[1:i, ], new_row, df[(i+1):nrow(df), ])
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
        
    } else if (sum_quan == target) {
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
    }
  }
}

df

In [ ]:
# Assuming df is your dataframe and vec is your vector
df <- data.frame(quan = c(1,2,3,4,5,6,7,8,9,10), col2 = letters[1:10], col3 = LETTERS[1:10])
vec <- c(6, 15, 28)

# Initialize variables
start_index <- 1
vec_index <- 1

# Loop over the vector values
for (target in vec) {
  # Initialize sum
  sum_quan <- 0
  
  # Loop over the dataframe rows
  for (i in start_index:nrow(df)) {
    # Add the current 'quan' value to the sum
    sum_quan <- sum_quan + df$quan[i]
    
    # Check if the sum exceeds the current target value
    if (sum_quan > target) {
      # Calculate the overage value
      overage <- sum_quan - target
      
      # Adjust the 'quan' value in the current row
      df$quan[i] <- df$quan[i] - overage
      
      # Create a new row with the overage value
      new_row <- df[i, ]
      new_row$quan <- overage
      df <- rbind(df[1:i, ], new_row, df[(i+1):nrow(df), ])
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 2
      
      # Break the loop as the target is reached
      break
    } else if (sum_quan == target) {
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
    }
  }
}

In [ ]:
# Demonstrates targeting ... but exceeding the index value again

vec <- c(1000,1000,1000,100,900,3000,6000,1000,1850,3000,3000,500,
           2000,3000,500,2000,810,1000,190,3000,1000,1100)

df <- read.csv("TestAllocate.csv", header = TRUE, sep = ",")


# Initialize variables
start_index <- 1
vec_index <- 1

# Create an empty dataframe to store the matches
matches <- data.frame()

# Loop over the vector values
for (target in vec) {
  # Initialize sum
  sum_quan <- 0
  
  # Loop over the dataframe rows
  for (i in start_index:nrow(df)) {
    # Add the current 'quan' value to the sum
    sum_quan <- sum_quan + df$quan[i]
    
    # Check if the sum exceeds the current target value
    if (sum_quan > target) {
      # Calculate the overage value
      overage <- sum_quan - target
      
      # Adjust the 'quan' value in the current row
      df$quan[i] <- df$quan[i] - overage
      
      # Add the current row to the matches dataframe
      matches <- rbind(matches, df[i, ])
      
      # Add a new row with the overage value to the matches dataframe
      matches <- rbind(matches, c(overage, df$col2[i], df$col3[i]))
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
        
    } else if (sum_quan == target) {
      # Add the current row to the matches dataframe
      matches <- rbind(matches, df[i, ])
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
    }
  }
}
matches

In [ ]:
# Assuming df is your dataframe and vec is your vector
df <- data.frame(quan = c(1,2,3,4,5,6,7,8,9,10), col2 = letters[1:10], col3 = LETTERS[1:10])
vec <- c(6, 14, 28)

# Initialize variables
start_index <- 1
vec_index <- 1

# Create an empty dataframe to store the matches
matches <- data.frame()

# Loop over the vector values
for (target in vec) {
  # Initialize sum
  sum_quan <- 0
  
  # Loop over the dataframe rows
  for (i in start_index:nrow(df)) {
    # Add the current 'quan' value to the sum
    sum_quan <- sum_quan + df$quan[i]
    
    # Check if the sum exceeds the current target value
    if (sum_quan > target) {
      # Calculate the overage value
      overage <- sum_quan - target
      
      # Adjust the 'quan' value in the current row
      df$quan[i] <- df$quan[i] - overage
      
      # Add the current row to the matches dataframe
      matches <- rbind(matches, df[i, ])
      
      # Add a new row with the overage value to the matches dataframe
      matches <- rbind(matches, c(overage, df$col2[i], df$col3[i]))
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
    } else if (sum_quan == target) {
      # Add the current row to the matches dataframe
      matches <- rbind(matches, df[i, ])
      
      # Do some code here
      print(paste("Match found for target", target, "at row", i))
      
      # Update the start index for the next loop
      start_index <- i + 1
      
      # Break the loop as the target is reached
      break
    }
  }
}
matches

In [ ]:
f <- function(x) x + 1
str(f)
('--------------------------------------------')
result <- f(60)                                   # Current age
result
('--------------------------------------------')
str(result)

print(f)